# Introduction to GROVER

In this tutorial, we will go over what Grover is, and how to get it up and running.

GROVER, or, Graph Representation frOm selfsuperVised mEssage passing tRansformer, is a novel framework proposed by Tencent AI Lab. GROVER utilizes self-supervised tasks in the node, edge and graph level in order to learn rich structural and semantic information of molecules from large unlabelled molecular datasets. GROVER integrates Message Passing Networks into a Transformer-style architecture to deliver more expressive molecular encoding.

Reference Paper: [Rong, Yu, et al. "Grover: Self-supervised message passing transformer on large-scale molecular data." Advances in Neural Information Processing Systems (2020).](https://drug.ai.tencent.com/publications/GROVER.pdf)

## Colab

This tutorial and the rest in this sequence are designed to be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Introduction_to_GROVER.ipynb)

## Setup

To run DeepChem within Colab, you'll need to run the following installation commands. This will take about 5 minutes to run to completion and install your environment. You can of course run this tutorial locally if you prefer. In that case, don't run these cells since they will download and install Anaconda on your local machine.

## Import and Setup required modules.
We will first clone the repository onto the preferred platform, then install it as a library. We will also import deepchem and install descriptastorus.

NOTE: The [original GROVER repository](https://github.com/tencent-ailab/grover) does not contain a `setup.py` file, thus we are currently using a fork which does.

In [ ]:
# Clone the forked repository.
%cd drive/MyDrive
!git clone https://github.com/atreyamaj/grover.git

/content/drive/MyDrive
fatal: destination path 'grover' already exists and is not an empty directory.


In [ ]:
# Navigate to the working folder.
%cd grover

/content/drive/MyDrive/grover


In [ ]:
# Install the forked repository.
!pip install -e ./

Obtaining file:///content/drive/MyDrive/grover
  Running setup.py develop for grover


In [ ]:
!git clone https://github.com/bp-kelley/descriptastorus

fatal: destination path 'descriptastorus' already exists and is not an empty directory.


In [ ]:
# Install deepchem and descriptastorus.
!pip install deepchem


In [ ]:
!pip install /content/grover/descriptastorus

Processing ./descriptastorus
  Preparing metadata (setup.py) ... done
  Created wheel for descriptastorus: filename=descriptastorus-2.5.0.23-py3-none-any.whl size=1083537 sha256=e9afdd74b9f765e179cbac608caeedaf4d550c4a492471bf1add8fa953dad68e
  Stored in directory: /tmp/pip-ephem-wheel-cache-wd_tv_45/wheels/33/a7/79/a735914259445b357d6b62ac74806fd7d62f69c1305e4dd5fe
Successfully built descriptastorus
  Attempting uninstall: descriptastorus
    Found existing installation: descriptastorus 2.5.0.23
    Uninstalling descriptastorus-2.5.0.23:
      Successfully uninstalled descriptastorus-2.5.0.23


## Extracting semantic motif labels
The semantic motif label is extracted by `scripts/save_feature.py` with feature generator `fgtasklabel`.

In [ ]:
!python scripts/save_features.py --data_path exampledata/pretrain/tryout.csv  \
                                --save_path exampledata/pretrain/tryout.npz   \
                                --features_generator fgtasklabel \
                                --restart

100% 5970/5970 [00:12<00:00, 472.47it/s]


## Extracting atom/bond contextual properties (vocabulary)
The atom/bond Contextual Property (Vocabulary) is extracted by `scripts/build_vocab.py`.

In [ ]:
!python scripts/build_vocab.py --data_path exampledata/pretrain/tryout.csv  \
                             --vocab_save_folder exampledata/pretrain  \
                             --dataset_name tryout

Building atom vocab from file: exampledata/pretrain/tryout.csv
50000it [00:03, 12577.14it/s]
atom vocab size 324
Building bond vocab from file: exampledata/pretrain/tryout.csv
50000it [00:21, 2334.97it/s]
bond vocab size 353


## Splitting the data
To accelerate the data loading and reduce the memory cost in the multi-gpu pretraining scenario, the unlabelled molecular data need to be spilt into several parts using `scripts/split_data.py`.

In [ ]:
!python scripts/split_data.py --data_path exampledata/pretrain/tryout.csv  \
                             --features_path exampledata/pretrain/tryout.npz  \
                             --sample_per_file 100  \
                             --output_path exampledata/pretrain/tryout

Number of files: 60


## Running Pretraining on Single GPU

In [ ]:
!python main.py pretrain \
               --data_path exampledata/pretrain/tryout \
               --save_dir model/tryout \
               --atom_vocab_path exampledata/pretrain/tryout_atom_vocab.pkl \
               --bond_vocab_path exampledata/pretrain/tryout_bond_vocab.pkl \
               --batch_size 32 \
               --dropout 0.1 \
               --depth 5 \
               --num_attn_head 1 \
               --hidden_size 100 \
               --epochs 3 \
               --init_lr 0.0002 \
               --max_lr 0.0004 \
               --final_lr 0.0001 \
               --weight_decay 0.0000001 \
               --activation PReLU \
               --backbone gtrans \
               --embedding_output_type both

[WARNING] Horovod cannot be imported; multi-GPU training is unsupported
Namespace(parser_name='pretrain', cuda=True, enable_multi_gpu=False, data_path='exampledata/pretrain/tryout', fg_label_path=None, atom_vocab_path='exampledata/pretrain/tryout_atom_vocab.pkl', bond_vocab_path='exampledata/pretrain/tryout_bond_vocab.pkl', embedding_output_type='both', save_dir='model/tryout', save_interval=9999999999, hidden_size=100, bias=False, depth=5, dropout=0.1, activation='PReLU', undirected=False, weight_decay=1e-07, num_attn_head=1, num_mt_block=1, dist_coff=0.1, backbone='gtrans', epochs=3, batch_size=32, warmup_epochs=2.0, init_lr=0.0002, max_lr=0.0004, final_lr=0.0001, bond_drop_rate=0, dense=False, fine_tune_coff=1, no_cache=True)
Loading data
Loading data:
Number of files: 60
Number of samples: 5970
Samples/file: 100
Splitting data with seed 0.
Total size = 5,970 | train size = 5,400 | val size = 570
atom vocab size: 324, bond vocab size: 353, Number of FG tasks: 85
Pre-loaded test data

# Training and Finetuning

##Extracting Molecular Features

Given a labelled molecular dataset, it is possible to extract the additional molecular features in order to train & finetune the model from the existing pretrained model. The feature matrix is stored as `.npz`.

In [ ]:
!python scripts/save_features.py --data_path exampledata/finetune/hERG.csv \
                                --save_path exampledata/finetune/hERG.npz \
                                --features_generator rdkit_2d_normalized \
                                --restart

100% 2968/2968 [03:13<00:00, 15.31it/s]


## Finetuning with existing data
Given the labelled dataset and the molecular features, we can use `finetune` function to finetune the pretrained model.

In [ ]:
!python main.py finetune --data_path exampledata/finetune/hERG.csv \
                        --features_path exampledata/finetune/hERG.npz \
                        --save_dir model/finetune/hERG/ \
                        --checkpoint_path model/tryout/model.ep3 \
                        --dataset_type classification \
                        --split_type scaffold_balanced \
                        --ensemble_size 1 \
                        --num_folds 3 \
                        --no_features_scaling \
                        --ffn_hidden_size 200 \
                        --batch_size 32 \
                        --epochs 10 \
                        --init_lr 0.00015

[WARNING] Horovod cannot be imported; multi-GPU training is unsupported
Fold 0
Loading data
Number of tasks = 1
Splitting data with seed 0
100% 2968/2968 [00:02<00:00, 1181.31it/s]
Total scaffolds = 1,422 | train scaffolds = 1,106 | val scaffolds = 122 | test scaffolds = 194
Label averages per scaffold, in decreasing order of scaffold frequency,capped at 10 scaffolds and 20 labels: [(array([1.]), array([4])), (array([1.]), array([1])), (array([1.]), array([1])), (array([0.]), array([1])), (array([1.]), array([1])), (array([0.]), array([1])), (array([1.]), array([1])), (array([0.]), array([2])), (array([0.]), array([1])), (array([0.]), array([2]))]
Class sizes
Activity 0: 35.58%, 1: 64.42%
Total size = 2,968 | train size = 2,374 | val size = 296 | test size = 298
Loading model 0 from model/tryout/model.ep3
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_q.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_q.W_h.weight".
Load

# Predicting output

## Extracting molecular features

If the finetuned model uses the molecular feature as input, we need to generate the molecular feature for the target molecules as well.

In [ ]:
!python scripts/save_features.py --data_path exampledata/finetune/hERG.csv \
                                --save_path exampledata/finetune/hERG.npz \
                                --features_generator rdkit_2d_normalized \
                                --restart

100% 2968/2968 [03:08<00:00, 15.73it/s]


## Predicting output with the finetuned model

In [ ]:
!python main.py predict --data_path exampledata/finetune/hERG.csv \
               --features_path exampledata/finetune/hERG.npz \
               --checkpoint_dir ./model \
               --no_features_scaling \
               --output data_pre.csv

[WARNING] Horovod cannot be imported; multi-GPU training is unsupported
Loading training args
Loading data
Validating SMILES
Test size = 2,968
Predicting...
  0% 0/6 [00:00<?, ?it/s]Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_q.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_q.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_k.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_k.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_v.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_v.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.layernorm.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.layernorm.bias".
Loading pretrained parameter "grover.encoders.e

## Output

The output will be saved in a file called `data_pre.csv`.

In [ ]:
import pandas as pd
df = pd.read_csv("/content/grover/data_pre.csv")
df.head(10)

,Unnamed: 0,Activity
0,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,0.937507
1,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,0.901423
2,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,0.865107
3,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,0.893256
4,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,0.857232
5,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,0.937507
6,COC(=O)[C@H]1[C@@H](OC(=O)c2ccccc2)C[C@@H]2CC[...,0.527712
7,O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1,0.900057
8,COc1ccc(Cl)cc1C(=O)NCCc1ccc(S(=O)(=O)NC(=O)NC2...,0.628845
9,CCCc1nn(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(C)...,0.364751
